In [1]:
import sys
import os

# Allow notebook to find `src/` folder for imports
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))


In [2]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

import src.config as c
print("Prediction FG version:", c.FEATURE_GROUP_MODEL_PREDICTION_VERSION)


Prediction FG version: 2


In [3]:
from datetime import datetime, timedelta
import pandas as pd
import logging, sys
import hopsworks

from src.data_utils import (
    load_and_process_citibike_data_from_local,
    transform_raw_data_into_ts_data,
)
import src.config as config
from hsfs.feature import Feature


In [4]:
from src.data_utils import load_and_process_citibike_data_from_local

In [6]:
# at the top of your notebook
from src.data_utils import load_and_process_citibike_data_from_local


In [7]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s  %(levelname)s  %(message)s", handlers=[logging.StreamHandler(sys.stdout)])
logger = logging.getLogger(__name__)

logger.info("📅 Loading Citi Bike data from Jan 2023 to Mar 2025 …")

raw_rides_2023 = load_and_process_citibike_data_from_local(
    year=2023,
    months=list(range(1, 13)),
    base_path=config.LOCAL_CITIBIKE_DATA_PATH
)

raw_rides_2024 = load_and_process_citibike_data_from_local(
    year=2024,
    months=list(range(1, 13)),
    base_path=config.LOCAL_CITIBIKE_DATA_PATH
)

raw_rides_2025 = load_and_process_citibike_data_from_local(
    year=2025,
    months=[1, 2, 3],
    base_path=config.LOCAL_CITIBIKE_DATA_PATH
)

raw_rides = pd.concat([raw_rides_2023, raw_rides_2024, raw_rides_2025])
logger.info(f"✅ Total rows loaded: {len(raw_rides):,}")

print(raw_rides["pickup_location_id"].unique())

print(raw_rides.head())

2025-05-03 22:24:46,263 INFO: 📅 Loading Citi Bike data from Jan 2023 to Mar 2025 …


2025-05-03 22:27:12,480 INFO: ✅ Total rows loaded: 799,733
['5329.03' '6140.05' '6948.10']
            ride_id  rideable_type         pickup_datetime  \
0  A46D077151843D7B   classic_bike 2023-01-16 10:39:54.386   
1  233875BAED2E02D0   classic_bike 2023-01-12 16:55:30.755   
2  8DD222EA1A1B0BC9  electric_bike 2023-01-08 19:32:25.647   
3  58976A4F584F8D28   classic_bike 2023-01-27 20:01:52.897   
4  FDD4C1E89A26727C   classic_bike 2023-01-13 18:02:38.160   

                  ended_at     start_station_name pickup_location_id  \
0  2023-01-16 10:45:18.005  West St & Chambers St            5329.03   
1  2023-01-12 17:04:03.688  West St & Chambers St            5329.03   
2  2023-01-08 19:42:00.382  West St & Chambers St            5329.03   
3  2023-01-27 20:08:58.118  West St & Chambers St            5329.03   
4  2023-01-13 18:11:22.139  West St & Chambers St            5329.03   

  end_station_name end_station_id  start_lat  start_lng    end_lat    end_lng  \
0   West Thames St    

In [8]:
TOP_3_IDS = {"6140.05", "6948.10", "5329.03"}

raw_rides_top3 = raw_rides[raw_rides["pickup_location_id"].astype(str).isin(TOP_3_IDS)].copy()
logger.info(f"📉 Filtered for top 3 stations: {len(raw_rides_top3):,} rows")


2025-05-03 22:28:09,095 INFO: 📉 Filtered for top 3 stations: 799,733 rows


In [9]:
raw_rides_top3.head()

,ride_id,rideable_type,pickup_datetime,ended_at,start_station_name,pickup_location_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,source_file,Unnamed: 0,rideable_type_duplicate_column_name_1
0,A46D077151843D7B,classic_bike,2023-01-16 10:39:54.386,2023-01-16 10:45:18.005,West St & Chambers St,5329.03,West Thames St,5114.06,40.717548,-74.013221,40.708347,-74.017134,member,202301-citibike-tripdata_1.csv,NaN,NaN
1,233875BAED2E02D0,classic_bike,2023-01-12 16:55:30.755,2023-01-12 17:04:03.688,West St & Chambers St,5329.03,West Thames St,5114.06,40.717548,-74.013221,40.708347,-74.017134,member,202301-citibike-tripdata_1.csv,NaN,NaN
2,8DD222EA1A1B0BC9,electric_bike,2023-01-08 19:32:25.647,2023-01-08 19:42:00.382,West St & Chambers St,5329.03,West Thames St,5114.06,40.717618,-74.013071,40.708347,-74.017134,member,202301-citibike-tripdata_1.csv,NaN,NaN
3,58976A4F584F8D28,classic_bike,2023-01-27 20:01:52.897,2023-01-27 20:08:58.118,West St & Chambers St,5329.03,West Thames St,5114.06,40.717548,-74.013221,40.708347,-74.017134,member,202301-citibike-tripdata_1.csv,NaN,NaN
4,FDD4C1E89A26727C,classic_bike,2023-01-13 18:02:38.160,2023-01-13 18:11:22.139,West St & Chambers St,5329.03,West Thames St,5114.06,40.717548,-74.013221,40.708347,-74.017134,member,202301-citibike-tripdata_1.csv,NaN,NaN


In [10]:
logger.info("🧮 Aggregating to hourly time-series format …")
ts_data = transform_raw_data_into_ts_data(raw_rides_top3)
logger.info(f"✅ Transformed data shape: {ts_data.shape}")
ts_data.head()


2025-05-03 22:28:19,876 INFO: 🧮 Aggregating to hourly time-series format …
2025-05-03 22:28:20,114 INFO: ✅ Transformed data shape: (59367, 3)


,pickup_hour,pickup_location_id,rides
0,2022-12-28 11:00:00,5329.03,1
1,2022-12-28 12:00:00,5329.03,0
2,2022-12-28 13:00:00,5329.03,0
3,2022-12-28 14:00:00,5329.03,0
4,2022-12-28 15:00:00,5329.03,0


In [9]:
logger.info("🔐 Logging in to Hopsworks …")
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY,
)
fs = project.get_feature_store()


2025-05-03 18:34:25,666 INFO: 🔐 Logging in to Hopsworks …
2025-05-03 18:34:25,671 INFO: Initializing external client
2025-05-03 18:34:25,671 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-03 18:34:26,600 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214683


In [10]:
from hsfs.feature import Feature

fg_schema = [
    Feature("pickup_hour",        "timestamp"),
    Feature("pickup_location_id", "string"),
    Feature("rides",              "int"),
]

logger.info("📦 Writing to Hopsworks feature group …")
hourly_fg = fs.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Hourly Citi Bike rides per location (2023–2025)",
    primary_key=["pickup_hour", "pickup_location_id"],
    event_time="pickup_hour",
    online_enabled=False,
    features=fg_schema,
)

ts_data["pickup_location_id"] = ts_data["pickup_location_id"].astype(str)
ts_data["rides"] = ts_data["rides"].astype("int32")

hourly_fg.insert(ts_data, write_options={"wait_for_job": True})
logger.info("✅ Done uploading data to Hopsworks!")


2025-05-03 18:35:12,884 INFO: 📦 Writing to Hopsworks feature group …
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1214683/fs/1202314/fg/1452362


Uploading Dataframe: 100.00% |██████████| Rows 59367/59367 | Elapsed Time: 00:04 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214683/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions
2025-05-03 18:35:28,870 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-05-03 18:35:31,961 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-05-03 18:37:54,008 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-05-03 18:37:54,075 INFO: Waiting for log aggregation to finish.
2025-05-03 18:38:02,389 INFO: Execution finished successfully.
2025-05-03 18:38:02,392 INFO: ✅ Done uploading data to Hopsworks!


In [11]:
logger.info("🔎 Creating Feature View …")

from hsfs.feature import Feature

feature_group = fs.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION
)

feature_view = fs.get_or_create_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION,
    description="Feature view for Citi Bike hourly demand",
    labels=[],
    query=feature_group.select_all()
)

logger.info("✅ Feature View created successfully.")


2025-05-03 18:43:29,993 INFO: 🔎 Creating Feature View …
Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1214683/fs/1202314/fv/time_series_hourly_feature_view/version/1
2025-05-03 18:43:31,954 INFO: ✅ Feature View created successfully.


In [11]:
from src.data_utils import transform_ts_data_info_features_and_target_loop

logger.info("📊 Generating sliding window features …")

# Use correct window size
window_size = 672

# Get full transformed DataFrame with all metadata
features_df, _ = transform_ts_data_info_features_and_target_loop(
    ts_data, feature_col="rides", window_size=window_size, step_size=1
)

# Check shape and available columns
print("✅ Transformed shape:", features_df.shape)
print("✅ Sample columns:", features_df.columns[:5].tolist() + ["...", features_df.columns[-3], features_df.columns[-2], features_df.columns[-1]])


2025-05-03 22:32:52,378 INFO: 📊 Generating sliding window features …
✅ Transformed shape: (57351, 675)
✅ Sample columns: ['rides_t-672', 'rides_t-671', 'rides_t-670', 'rides_t-669', 'rides_t-668', '...', 'target', 'pickup_hour', 'pickup_location_id']


In [12]:
features_df.head()

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,target,pickup_hour,pickup_location_id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,5,12,15,12,4,6,2023-01-25 11:00:00,5329.03
1,0,0,0,0,0,0,0,0,0,0,...,0,5,12,15,12,4,6,2,2023-01-25 12:00:00,5329.03
2,0,0,0,0,0,0,0,0,0,0,...,5,12,15,12,4,6,2,5,2023-01-25 13:00:00,5329.03
3,0,0,0,0,0,0,0,0,0,0,...,12,15,12,4,6,2,5,2,2023-01-25 14:00:00,5329.03
4,0,0,0,0,0,0,0,0,0,0,...,15,12,4,6,2,5,2,0,2023-01-25 15:00:00,5329.03


In [18]:
# 0️⃣ load the full Pipeline
import os, sys, joblib

model_path = os.path.abspath(
    os.path.join(os.getcwd(), "..", "models", "lgb_model.pkl")
)
lgb_pipeline = joblib.load(model_path)

# 1️⃣ pull out the underlying Booster and its feature‐name list
#   (depends on whether you have a plain LGBMModel or a sklearn Pipeline)
if hasattr(lgb_pipeline, "booster_"):
    booster = lgb_pipeline.booster_
elif hasattr(lgb_pipeline, "_Booster"):
    booster = lgb_pipeline._Booster
else:
    # assume a Pipeline and last step is the LGB
    booster = lgb_pipeline.steps[-1][1].booster_

expected = booster.feature_name()
print("➜ Trained model expected", len(expected), "features")
print("First 5:", expected[:5])
print("Last 5: ", expected[-5:])

# 2️⃣ build your X exactly as you have it now
lag_cols   = [c for c in features_df.columns if c.startswith("rides_t-")]
input_cols = lag_cols + ["pickup_hour", "pickup_location_id"]
X = features_df[input_cols]

print("➜ Your X has", X.shape[1], "columns.")
print("First 5:", X.columns[:5].tolist())
print("Last 5: ", X.columns[-5:].tolist())

# 3️⃣ see which names don’t line up
missing_in_X   = set(expected) - set(X.columns)
extra_in_X     = set(X.columns) - set(expected)
print("✖ Features expected by model but missing in X:", sorted(missing_in_X))
print("✖ Extra columns in X that the model didn’t expect:", sorted(extra_in_X))


➜ Trained model expected 676 features
First 5: ['rides_t-672', 'rides_t-671', 'rides_t-670', 'rides_t-669', 'rides_t-668']
Last 5:  ['rides_t-1', 'target', 'average_rides_last_4_weeks', 'hour', 'day_of_week']
➜ Your X has 674 columns.
First 5: ['rides_t-672', 'rides_t-671', 'rides_t-670', 'rides_t-669', 'rides_t-668']
Last 5:  ['rides_t-3', 'rides_t-2', 'rides_t-1', 'pickup_hour', 'pickup_location_id']
✖ Features expected by model but missing in X: ['average_rides_last_4_weeks', 'day_of_week', 'hour', 'target']
✖ Extra columns in X that the model didn’t expect: ['pickup_hour', 'pickup_location_id']


In [23]:
import os, sys, joblib
import pandas as pd

# 1️⃣ Locate & load your full pipeline (with featurizer + LGB):
model_path = os.path.abspath(
    os.path.join(os.getcwd(), "..", "models", "lgb_model.pkl")
)
pipeline = joblib.load(model_path)

# 2️⃣ Build the raw X exactly as the pipeline saw it at train time:
lag_cols = [c for c in features_df.columns if c.startswith("rides_t-")]

X_raw = features_df[ 
    lag_cols
    + ["target", "pickup_hour", "pickup_location_id"]
].copy()

# 3️⃣ Now run it:
preds = pipeline.predict(X_raw)

# 4️⃣ Stick them back:
features_df = features_df.copy()
features_df["predicted_rides"] = preds

features_df[["pickup_hour", "pickup_location_id", "predicted_rides"]].head()


,pickup_hour,pickup_location_id,predicted_rides
0,2023-01-25 11:00:00,5329.03,5.999306
1,2023-01-25 12:00:00,5329.03,1.998759
2,2023-01-25 13:00:00,5329.03,4.998313
3,2023-01-25 14:00:00,5329.03,1.998759
4,2023-01-25 15:00:00,5329.03,0.001673


In [25]:
import os, sys, joblib
import pandas as pd

# 0️⃣ If you haven’t already, make sure your notebook can see ../models
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

# 1️⃣ Load the full sklearn Pipeline you trained
model_path = os.path.join(os.getcwd(), "..", "models", "lgb_model.pkl")
assert os.path.exists(model_path), f"Model not found: {model_path}"
pipeline = joblib.load(model_path)

# 2️⃣ Build the raw X exactly as your pipeline saw it at train time:
#    - ALL lag features ("rides_t-...") 
#    - the placeholder "target" column (pipeline will drop it internally)
#    - pickup_hour (for hour/day features)
#    - pickup_location_id (so the pipeline’s drop-step can find it)
lag_cols = [col for col in features_df.columns if col.startswith("rides_t-")]
X_raw   = features_df[lag_cols + ["target", "pickup_hour", "pickup_location_id"]].copy()

# 3️⃣ Run the pipeline’s predict (it will extend with avg/day/hour, drop extras, then LGBM)
preds = pipeline.predict(X_raw)

# 4️⃣ Round to int32 (since your FG schema is `int`)
features_df = features_df.copy()  # avoid any warning
features_df["predicted_rides"] = preds.round(0).astype("int32")

# 5️⃣ Quick sanity check
features_df[["pickup_hour", "pickup_location_id", "predicted_rides"]].head()


,pickup_hour,pickup_location_id,predicted_rides
0,2023-01-25 11:00:00,5329.03,6
1,2023-01-25 12:00:00,5329.03,2
2,2023-01-25 13:00:00,5329.03,5
3,2023-01-25 14:00:00,5329.03,2
4,2023-01-25 15:00:00,5329.03,0


In [26]:
features_df.shape

(57351, 676)

In [27]:
# quick sanity‑check
print(features_df[["pickup_hour", "pickup_location_id", "predicted_rides"]].head())


          pickup_hour pickup_location_id  predicted_rides
0 2023-01-25 11:00:00            5329.03                6
1 2023-01-25 12:00:00            5329.03                2
2 2023-01-25 13:00:00            5329.03                5
3 2023-01-25 14:00:00            5329.03                2
4 2023-01-25 15:00:00            5329.03                0


In [28]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

import src.config as c
print("Prediction FG version:", c.FEATURE_GROUP_MODEL_PREDICTION_VERSION)


Prediction FG version: 2


In [29]:
import hopsworks
from hsfs.feature import Feature
import src.config as c

# 1️⃣ Log in
project = hopsworks.login(
    project       = c.HOPSWORKS_PROJECT_NAME,
    api_key_value = c.HOPSWORKS_API_KEY,
)
fs = project.get_feature_store()

# 2️⃣ Get-or-create the prediction Feature Group v2
pred_fg = fs.get_or_create_feature_group(
    name          = c.FEATURE_GROUP_MODEL_PREDICTION,
    version       = c.FEATURE_GROUP_MODEL_PREDICTION_VERSION,
    description   = "Next-hour demand predictions from LGBM model (v2)",
    primary_key   = ["pickup_location_id", "pickup_hour"],
    event_time    = "pickup_hour",
    online_enabled=False,
    features      = [
        Feature("pickup_location_id", "string"),
        Feature("pickup_hour",        "timestamp"),
        Feature("predicted_rides",    "int"),
    ],
)
print(f"✅ Using Feature Group “{pred_fg.name}” v{pred_fg.version} (id {pred_fg.id})")

# 3️⃣ Prepare your output DataFrame exactly to schema
out_df = features_df[["pickup_hour", "pickup_location_id", "predicted_rides"]].copy()
out_df["pickup_location_id"] = out_df["pickup_location_id"].astype(str)
out_df["predicted_rides"]    = out_df["predicted_rides"].round(0).astype("int32")

# 4️⃣ Insert & wait for completion
pred_fg.insert(out_df, write_options={"wait_for_job": True})
print("🚀  Predictions uploaded to Hopsworks!")


2025-05-03 23:03:33,604 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-03 23:03:33,608 INFO: Initializing external client
2025-05-03 23:03:33,609 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-03 23:03:34,570 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214683
✅ Using Feature Group “taxi_hourly_model_prediction” v2 (id None)
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1214683/fs/1202314/fg/1438379


Uploading Dataframe: 100.00% |██████████| Rows 57351/57351 | Elapsed Time: 00:04 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214683/jobs/named/taxi_hourly_model_prediction_2_offline_fg_materialization/executions
2025-05-03 23:03:52,947 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-05-03 23:03:56,042 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-05-03 23:06:34,135 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-05-03 23:06:34,214 INFO: Waiting for log aggregation to finish.
2025-05-03 23:06:42,586 INFO: Execution finished successfully.
🚀  Predictions uploaded to Hopsworks!
